In [ ]:
import matplotlib.pyplot as plt
import nest
import numpy as np
import os
from pynestml.frontend.pynestml_frontend import generate_nest_target

In [ ]:
def generate_code(neuron_model: str, models_path=""):
    """
    Generate NEST code for neuron model with gap junction support.
    Parameters
    ----------
    neuron_model : str
        Name of the neuron model to compile. This should correspond to a 
        .nestml file containing the neuron model definition.
    models_path : str, optional
        Path to the directory containing the NESTML model files.
        Default is empty string (current directory).
    """
    codegen_opts = {"gap_junctions": {"enable": True,
                                        "gap_current_port": "I_stim",
                                        "membrane_potential_variable": "V_m"}}

    files = os.path.join(models_path, neuron_model + ".nestml")
    generate_nest_target(input_path=files,
                            logging_level="WARNING",
                            module_name="nestml_gap_" + neuron_model + "_module",
                            suffix="_nestml",
                            codegen_opts=codegen_opts)

    return neuron_model

In [ ]:
def initialize_hh():
    nest.Install("nestml_gap_hh_psc_alpha_neuron_module")
    neuron = nest.Create("hh_psc_alpha_neuron_nestml", 2)
    neuron.I_e = 650.0
    neuron[0].V_m = -10.0

    return neuron

In [ ]:
def initialize_aeif():
    nest.Install("nestml_gap_aeif_cond_exp_neuron_module")
    neuron = nest.Create("aeif_cond_exp_neuron_nestml", 2)
    neuron.I_e = 650.0
    neuron[0].V_m = -10.0 

    return neuron

In [ ]:
def initilize_eglif():
    nest.Install("nestml_gap_eglif_cond_alpha_multisyn_module")
    neuron = nest.Create("eglif_cond_alpha_multisyn_nestml", 2)
    neuron[0].V_m = -46.0 

    return neuron

In [ ]:
def initialize_cells(model):
    if model == "hh":
        return initialize_hh()
    elif model == "aeif":
        return initialize_aeif()
    elif model == "eglif":
        return initilize_eglif()
        

In [ ]:
def plot_vm(vm):
    vm_values = vm.events["V_m"]
    senders = vm.events["senders"]
    times = vm.events["times"]
    plt.figure(figsize=(10, 5))
    plt.plot(
        times[np.where(senders == 1)], vm_values[np.where(senders == 1)], "r-",label="Neuron 1")
    plt.plot(
        times[np.where(senders == 2)], vm_values[np.where(senders == 2)], "g-", label="Neuron 2")
    plt.legend(loc='upper right')
    plt.xlabel("time (ms)")
    plt.ylabel("membrane potential (mV)")
    plt.show()

In [ ]:
def plot_mm_stim(mm_stim):
    I_stim_values = mm_stim.events["I_stim_recordable"]
    senders_Istim = mm_stim.events["senders"]
    times_Istim = mm_stim.events["times"]

    plt.figure(figsize=(10, 5))
    plt.plot(
        times_Istim[np.where(senders_Istim == 1)], I_stim_values[np.where(senders_Istim == 1)], "r-",label="Neuron 1"
    )
    plt.plot(
        times_Istim[np.where(senders_Istim == 2)], I_stim_values[np.where(senders_Istim == 2)], "g-", label="Neuron 2"
    )
    plt.legend(loc="upper right")
    plt.xlabel("time (ms)")
    plt.ylabel("I_gap (pA)")
    plt.title("Gap junction currents")
    plt.show()

In [ ]:
def simulate_network(selected_model, gap=False, dc_stim=False):
    nest.ResetKernel()
    nest.resolution = 0.05

    neuron = initialize_cells(selected_model)

    vm = nest.Create("voltmeter", params={"interval": 0.1})
    nest.Connect(vm, neuron, "all_to_all")

    if dc_stim:
        dc = nest.Create("dc_generator", params={"amplitude": 0.5})
        nest.Connect(dc, neuron[0], syn_spec={"weight": 1.0})
    
    I_stim_recordable ="I_stim_recordable" in nest.GetDefaults(models.get(selected_model))["recordables"]
    
    if I_stim_recordable:
        mm_stim = nest.Create("multimeter", {"record_from": ["I_stim_recordable"] })
        nest.Connect(mm_stim, neuron)

    if gap:
        nest.Connect(
                neuron, neuron, 
                {"rule": "all_to_all", "allow_autapses": False}, 
                {"synapse_model": "gap_junction", "weight": 5}
            )

    nest.Simulate(5000.0)
    plot_vm(vm)
    if I_stim_recordable:
        plot_mm_stim(mm_stim)
    

    return {
        "V_m": vm.events["V_m"], 
        "I_stim": mm_stim.events["I_stim_recordable"] if I_stim_recordable else None
    }
    

In [ ]:
def confront_simulations(simulation_1, simulation_2):
    V_m_1 = simulation_1["V_m"]
    V_m_2 = simulation_2["V_m"]

    I_stim_1 = simulation_1["I_stim"]
    I_stim_2 = simulation_2["I_stim"]

    V_m_equal = np.array_equal(V_m_1, V_m_2)
    if V_m_equal:
        print("Simulations have the same V_m trace")
    else:
        print("Simulations have different V_m traces")

    I_stim_equal = np.array_equal(I_stim_1, I_stim_2)
    if I_stim_equal:
        print("Simulations have the same I_stim current")
    else:
        print("Simulations have different I_stim_currents")        
    

In [ ]:
models = {
    "hh": "hh_psc_alpha_neuron_nestml",
    "aeif": "aeif_cond_exp_neuron_nestml", 
    "eglif": "eglif_cond_alpha_multisyn_nestml"
}
selected_model = "eglif"
generate_model = False 

if generate_model:
    generate_code(neuron_model=selected_code, models_path="../nest_models")

simulation_nogap = simulate_network(selected_model, gap=False)
simulation_gap = simulate_network(selected_model, gap=True)

In [ ]:
confront_simulations(simulation_nogap, simulation_gap)